# 10_04: Modeling COVID-19 data

In [1]:
import math
import collections
import dataclasses
import datetime

import numpy as np
import pandas as pd

import plotly.express as px

In [2]:
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
covid19 = pd.read_csv('covid19.csv.gz', parse_dates=['date'], dtype_backend='pyarrow')

In [4]:
covid19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156710 entries, 0 to 156709
Data columns (total 15 columns):
 #   Column                      Non-Null Count   Dtype          
---  ------                      --------------   -----          
 0   country                     156710 non-null  string[pyarrow]
 1   date                        156710 non-null  datetime64[ns] 
 2   continent                   156710 non-null  string[pyarrow]
 3   population                  156710 non-null  int64[pyarrow] 
 4   life_expectancy             151672 non-null  double[pyarrow]
 5   gdp_per_capita              148190 non-null  int64[pyarrow] 
 6   population_density          156710 non-null  double[pyarrow]
 7   median_age                  156710 non-null  double[pyarrow]
 8   extreme_poverty             137966 non-null  double[pyarrow]
 9   human_development_index     156710 non-null  double[pyarrow]
 10  hospital_beds_per_thousand  122630 non-null  double[pyarrow]
 11  percent_fully_vaccinated  

In [5]:
covid19['deaths_per_million'] = covid19.total_deaths / (covid19.population / 1.0e6)
covid19['excess_per_million'] = covid19.total_excess / (covid19.population / 1.0e6)

final = covid19.groupby('country').last()

In [6]:
final.deaths_per_million.mean(), final.deaths_per_million.std()

(1068.7827868419265, 1322.4723872957682)

In [7]:
px.histogram(final.deaths_per_million)

In [8]:
explanatory = ['population', 'gdp_per_capita', 'population_density', 'life_expectancy',
               'median_age', 'extreme_poverty', 'human_development_index',
               'hospital_beds_per_thousand', 'percent_fully_vaccinated']

In [9]:
fit = smf.ols('deaths_per_million ~ gdp_per_capita + human_development_index',
              data=final.reset_index()).fit()
np.sqrt(fit.mse_resid), fit.rsquared, fit.fvalue

(np.float64(1035.8498259744388),
 np.float64(0.4186242340060866),
 np.float64(30.602462272988774))

In [10]:
fit = smf.ols('deaths_per_million ~ extreme_poverty + percent_fully_vaccinated',
              data=final.reset_index()).fit()
np.sqrt(fit.mse_resid), fit.rsquared, fit.fvalue

(np.float64(1150.5248204650236),
 np.float64(0.3117750909864718),
 np.float64(17.89402843849054))

In [11]:
import itertools
list(itertools.combinations(explanatory, 2))

[('population', 'gdp_per_capita'),
 ('population', 'population_density'),
 ('population', 'life_expectancy'),
 ('population', 'median_age'),
 ('population', 'extreme_poverty'),
 ('population', 'human_development_index'),
 ('population', 'hospital_beds_per_thousand'),
 ('population', 'percent_fully_vaccinated'),
 ('gdp_per_capita', 'population_density'),
 ('gdp_per_capita', 'life_expectancy'),
 ('gdp_per_capita', 'median_age'),
 ('gdp_per_capita', 'extreme_poverty'),
 ('gdp_per_capita', 'human_development_index'),
 ('gdp_per_capita', 'hospital_beds_per_thousand'),
 ('gdp_per_capita', 'percent_fully_vaccinated'),
 ('population_density', 'life_expectancy'),
 ('population_density', 'median_age'),
 ('population_density', 'extreme_poverty'),
 ('population_density', 'human_development_index'),
 ('population_density', 'hospital_beds_per_thousand'),
 ('population_density', 'percent_fully_vaccinated'),
 ('life_expectancy', 'median_age'),
 ('life_expectancy', 'extreme_poverty'),
 ('life_expectanc

In [12]:
def getfit(final, expvars):
    return smf.ols('deaths_per_million ~ ' + '+'.join(expvars), data=final.reset_index()).fit()

In [13]:
getfit(final, ['gdp_per_capita']).fvalue

np.float64(32.292801582811855)

In [14]:
fvalues = {expvars: getfit(final, expvars).fvalue for nvars in range(1, len(explanatory) + 1)
                                                  for expvars in itertools.combinations(explanatory, nvars)}

In [15]:
bestvars = max(fvalues.keys(), key=fvalues.get)
bestvars

('median_age',)

In [16]:
getfit(final, bestvars).summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                   Results: Ordinary least squares
=====================================================================
Model:              OLS                Adj. R-squared:     0.442     
Dependent Variable: deaths_per_million AIC:                1548.5102 
Date:               2025-06-11 22:20   BIC:                1553.5754 
No. Observations:   93                 Log-Likelihood:     -772.26   
Df Model:           1                  F-statistic:        73.81     
Df Residuals:       91                 Prob (F-statistic): 2.26e-13  
R-squared:          0.448              Scale:              9.7628e+05
---------------------------------------------------------------------
                Coef.    Std.Err.    t    P>|t|    [0.025     0.975] 
---------------------------------------------------------------------
Intercept     -1476.9366 313.5244 -4.7108 0.0000 -2099.7143 -854.1589
median_age       91.1775  10.6127  8.5914 0.0000    70.0968  112.2583
---------------------------------------------------------------------
Omnibus:               54.973        Durbin-Watson:           2.356  
Prob(Omnibus):         0.000         Jarque-Bera (JB):        340.171
Skew:                  1.716         Prob(JB):                0.000  
Kurtosis:              11.718        Condition No.:           90     
=====================================================================
Notes:
[1] Standard Errors assume that the covariance matrix of the errors
is correctly specified.
"""

In [17]:
covid19['year'] = covid19.date.dt.year

In [18]:
def getyear(year):
    return covid19[covid19.year == year].groupby('country').last()

In [19]:
for year in range(2020, 2024):
    final_by_year = getyear(year)
    
    fvalues = {expvars: getfit(final_by_year, expvars).fvalue
               for nvars in range(1, len(explanatory) + 1)
               for expvars in itertools.combinations(explanatory, nvars)}

    bestvars = max(fvalues.keys(), key=fvalues.get)
    bestfit = getfit(final_by_year, bestvars)
    
    print(f'In {year}, the best model is {'+'.join(bestvars)} with f={bestfit.fvalue:.1f}, res={np.sqrt(bestfit.mse_resid):.1f}')

In 2020, the best model is human_development_index with f=39.9, res=392.2
In 2021, the best model is median_age with f=46.3, res=880.7
In 2022, the best model is median_age with f=66.7, res=975.0
In 2023, the best model is median_age with f=73.1, res=984.7


In [20]:
getfit(getyear(2020), ['human_development_index']).summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                       Results: Ordinary least squares
==============================================================================
Model:                 OLS                   Adj. R-squared:        0.297     
Dependent Variable:    deaths_per_million    AIC:                   1376.6525 
Date:                  2025-06-11 22:24      BIC:                   1381.7177 
No. Observations:      93                    Log-Likelihood:        -686.33   
Df Model:              1                     F-statistic:           39.91     
Df Residuals:          91                    Prob (F-statistic):    9.61e-09  
R-squared:             0.305                 Scale:                 1.5382e+05
------------------------------------------------------------------------------
                          Coef.   Std.Err.    t    P>|t|    [0.025     0.975] 
------------------------------------------------------------------------------
Intercept               -758.5526 172.8678 -4.3880 0.0000 -1101.9333 -415.1719
human_development_index 1537.2472 243.3227  6.3177 0.0000  1053.9165 2020.5779
------------------------------------------------------------------------------
Omnibus:                  77.092           Durbin-Watson:              2.341  
Prob(Omnibus):            0.000            Jarque-Bera (JB):           731.343
Skew:                     2.490            Prob(JB):                   0.000  
Kurtosis:                 15.804           Condition No.:              9      
==============================================================================
Notes:
[1] Standard Errors assume that the covariance matrix of the errors is
correctly specified.
"""

In [21]:
fit = smf.ols('excess_per_million ~ human_development_index', data=getyear(2020).reset_index()).fit()
fit.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                        Results: Ordinary least squares
================================================================================
Model:                   OLS                   Adj. R-squared:        -0.004    
Dependent Variable:      excess_per_million    AIC:                   741.0973  
Date:                    2025-06-11 22:25      BIC:                   744.7546  
No. Observations:        46                    Log-Likelihood:        -368.55   
Df Model:                1                     F-statistic:           0.8060    
Df Residuals:            44                    Prob (F-statistic):    0.374     
R-squared:               0.018                 Scale:                 5.5706e+05
--------------------------------------------------------------------------------
                          Coef.     Std.Err.    t    P>|t|    [0.025     0.975] 
--------------------------------------------------------------------------------
Intercept                1839.4950 1023.9519  1.7965 0.0793  -224.1444 3903.1345
human_development_index -1107.8743 1233.9978 -0.8978 0.3742 -3594.8335 1379.0849
--------------------------------------------------------------------------------
Omnibus:                   1.778             Durbin-Watson:                2.431
Prob(Omnibus):             0.411             Jarque-Bera (JB):             1.588
Skew:                      0.329             Prob(JB):                     0.452
Kurtosis:                  2.372             Condition No.:                19   
================================================================================
Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly
specified.
"""